In [1]:
from datasets import load_dataset
import pandas as pd
import json
from datasets import Dataset, DatasetDict, concatenate_datasets
import spacy
import joblib

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
def sample_print(dataset, col, subset, n=5):
    for i in range(n):
        print(dataset[col][i])

In [20]:
dataset = load_dataset("Atipico1/mrqa_masked", split="train")

Generating train split: 100%|██████████| 528863/528863 [00:02<00:00, 191790.19 examples/s]


In [22]:
df = pd.DataFrame(dataset)

In [32]:
subset_name = "TriviaQA-web"
sample_contexts = df[df.subset==subset_name].sample(1000).context.tolist()[:10]
for sample in sample_contexts:
    print(sample)
    print()

[DOC] [TLE] President Gerald FordPresident Gerald Ford [PAR] Donald Trump [PAR] Key Dates [PAR] 1913 Born in Omaha, Nebraska. (His name then was Leslie Lynch King Jr. In 1916 he was adopted by his step father and his name was changed. [PAR] 1935 Gerald Ford graduated from the University of Michigan. [PAR] 1941 Ford graduated from the Yale Law School. [PAR] 1942 -46 He served in the Navy during WWII [PAR] 1948 Gerald Ford was elected to the U.S. House of Representatives. [PAR] 1964 Warren Commission which Ford served as a member concluded that there was no evidence of a conspiracy in the assassination of Kennedy. [PAR] 1972 Watergate break-in. [PAR] 1973 President Nixon appointed Ford as his Vice President. [PAR] 1974 Nixon resigned as President and Gerald Ford became President. [PAR] 1974 Ford pardoned Nixon. [PAR] 1976 Lost the presidential election to Jimmy Carter. [PAR] 1979 Ford published his memoir A Time to Heal. [PAR] 1980 Ford declined to serve as Reagan's vice president. [PAR]

In [17]:
df = joblib.load("/data/seongilpark/mrqa_df.joblib")

In [19]:
context = list(set(df["norm_context"]))
Dataset.from_dict({"context":context}).push_to_hub("Atipico1/unique_contexts")

Uploading the dataset shards: 100%|██████████| 2/2 [01:03<00:00, 31.61s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/unique_contexts/commit/cd1fe65dce4ce9b29a47c88b75dedf0e0865df26', commit_message='Upload dataset', commit_description='', oid='cd1fe65dce4ce9b29a47c88b75dedf0e0865df26', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
contexts = list(set(df.norm_context.tolist()))
len(contexts)

354604

In [6]:
ctx2doc = dict()

In [8]:
from tqdm.auto import tqdm
BATCH_SIZE = 500
for i in tqdm(range(0, len(contexts), BATCH_SIZE)):
    batch = contexts[i:i+BATCH_SIZE]
    docs = list(nlp.pipe(batch, batch_size=BATCH_SIZE))
    for ctx, doc in zip(batch, docs):
        ctx2doc[ctx] = {"doc":doc}

  0%|          | 0/710 [00:00<?, ?it/s]

  0%|          | 2/710 [00:52<5:07:36, 26.07s/it]


KeyboardInterrupt: 

In [2]:
mrqa = load_dataset("mrqa")
train, test, valid = mrqa["train"], mrqa["test"], mrqa["validation"]
total = concatenate_datasets([train, test, valid])

In [4]:
subset = total.remove_columns(["context_tokens", "qid", "question_tokens", "detected_answers"])
subset

Dataset({
    features: ['subset', 'context', 'question', 'answers'],
    num_rows: 584673
})

In [6]:
from src.utils import normalize_answer
normalize_answer("What are you Doing?? ")

'what are you doing'

In [5]:
df = pd.DataFrame(subset)

In [7]:
df["norm_q"] = df["question"].apply(normalize_answer)

In [20]:
df["subset"].value_counts().to_frame()

,count
subset,
SearchQA,134364
NaturalQuestionsShort,116907
SQuAD,97095
HotpotQA,78829
NewsQA,78372
TriviaQA-web,69473
RelationExtraction,2948
BioASQ,1504
DROP,1503


In [9]:
sub_df = df.drop_duplicates(subset=["norm_q"])

In [10]:
questions = sub_df.question.tolist()


/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [13]:
from tqdm.auto import tqdm
from preprocess import masking
BATCH_SIZE = 2000
result = []
for i in tqdm(range(0, len(questions), BATCH_SIZE)):
    batch = questions[i : i + BATCH_SIZE]
    docs = list(nlp.pipe(batch, batch_size=BATCH_SIZE))
    masked_quries = [masking(doc) for doc in docs]
    result.extend(masked_quries)

100%|██████████| 284/284 [58:37<00:00, 12.38s/it]


In [26]:
new_df = sub_sub_df.drop(columns=["norm_q", "index"])
new_df

,subset,context,question,answers,masked_q
0,SQuAD,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,[Saint Bernadette Soubirous],To whom did the Virgin Mary allegedly appear i...
1,SQuAD,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,[a copper statue of Christ],What is in front of [MASK]?
2,SQuAD,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,[the Main Building],[MASK] at [MASK] is beside to which structure?
3,SQuAD,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,[a Marian place of prayer and reflection],What is the Grotto at [MASK]?
4,SQuAD,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,[a golden statue of the Virgin Mary],What sits on top of [MASK] at [MASK]?
...,...,...,...,...,...
528858,NaturalQuestionsShort,"<P> Throughout the novel , Junior shares his d...",where does junior want to go to find hope,[off the rez],where does junior want to go to find hope
528859,NaturalQuestionsShort,"<P> Acceding to the throne in 1888 , he dismis...",what happened to germany 's leader after ww1,"[he lost the support of the army , abdicated i...",what happened to [MASK] 's leader after [MASK]
528860,NaturalQuestionsShort,<Table> Michigan State -- Notre Dame football ...,when will notre dame played michigan state again,[2026],when will [MASK] played [MASK] again
528861,NaturalQuestionsShort,<P> Due to Eric 's departure from the show at ...,who does eric end up with in that 70s show,[Donna],who does [MASK] end up with in that [MASK] show


In [27]:
for i, row in new_df.iterrows():

0


In [30]:
dataset = Dataset.from_pandas(new_df)
dataset

Dataset({
    features: ['subset', 'context', 'question', 'answers', 'masked_q'],
    num_rows: 528863
})

In [31]:
dataset.push_to_hub("Atipico1/mrqa_masked")

Uploading the dataset shards: 100%|██████████| 3/3 [01:15<00:00, 25.07s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/mrqa_masked/commit/e25e80b71f2928e9701d9beab1abcd704f6886a5', commit_message='Upload dataset', commit_description='', oid='e25e80b71f2928e9701d9beab1abcd704f6886a5', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
id2ctxs = dict()
for i, row in new_df.iterrows():
    id2ctxs[i] = row["context"]
import joblib
joblib.dump(id2ctxs, "/data/seongilpark/id2ctxs.joblib")

['/data/seongilpark/id2ctxs.joblib']

In [78]:
import re

def preprocess_text(text, subset_name: str):
    """
    This function removes all text between [TLE] and [SEP] including the markers themselves.
    """
    if subset_name == "NewsQA":
        start = text.find("--")
        end = start + 2
        text = text[end:]
    if subset_name == "NaturalQuestionsShort":
        if "<Table>" in text:
            return None
    text = text.replace("<P>", "")
    text = text.replace("</P>", "")
    if text.strip().startswith("[PAR]"):
        start = text.find("[PAR]")
        end = start + len("[PAR]")
        text = text[end:]
    if text.strip().startswith("[DOC]"):
        start = text.find("[DOC]")
        end = start + len("[DOC]")
        text = text[end:]
    while "[TLE]" in text and "[SEP]" in text:
        start = text.find("[TLE]")
        end = text.find("[SEP]") + len("[SEP]")
        text = text[:start] + text[end:]
    # Replace multiple spaces with a single space
    text = re.sub('\n+', '', text)
    text = re.sub(' +', ' ', text)
    return text

# Example usage
example_text = "[PAR] [DOC] This is a test [TLE] remove this part\n\n\n [SEP] and \n\n this [TLE] remove this part [SEP] remains."
ex_text = "[PAR] [TLE] Die Rhöner Säuwäntzt [SEP] Die Rhöner Säuwäntzt are a Skiffle-Bluesband from Eichenzell-Lütter in Hessen, Germany."
result = preprocess_text(ex_text, "NewsQA")
print(result)

PAR] Die Rhöner Säuwäntzt are a Skiffle-Bluesband from Eichenzell-Lütter in Hessen, Germany.


In [73]:
for i, row in tqdm(new_df.iterrows()):
    new_df.loc[i, "norm_context"] = preprocess_text(row["context"], row["subset"])

528863it [01:31, 5763.61it/s]


In [74]:
new_df.norm_context.isna().sum()

12180

In [75]:
new_df = new_df.dropna(subset=["norm_context"])
new_df

,subset,context,question,answers,masked_q,norm_context
0,SQuAD,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,[Saint Bernadette Soubirous],To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha..."
1,SQuAD,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,[a copper statue of Christ],What is in front of [MASK]?,"Architecturally, the school has a Catholic cha..."
2,SQuAD,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,[the Main Building],[MASK] at [MASK] is beside to which structure?,"Architecturally, the school has a Catholic cha..."
3,SQuAD,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,[a Marian place of prayer and reflection],What is the Grotto at [MASK]?,"Architecturally, the school has a Catholic cha..."
4,SQuAD,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,[a golden statue of the Virgin Mary],What sits on top of [MASK] at [MASK]?,"Architecturally, the school has a Catholic cha..."
...,...,...,...,...,...,...
528855,NaturalQuestionsShort,"<P> In social relationships , an ex ( plural i...",what is the meaning of x girl friend,[a former sexual or romantic partner],what is the meaning of x girl friend,"In social relationships , an ex ( plural is e..."
528857,NaturalQuestionsShort,<P> The film follows two families spending tim...,where does the great outdoors movie take place,[Wisconsin],where does the great outdoors movie take place,The film follows two families spending time o...
528858,NaturalQuestionsShort,"<P> Throughout the novel , Junior shares his d...",where does junior want to go to find hope,[off the rez],where does junior want to go to find hope,"Throughout the novel , Junior shares his drea..."
528859,NaturalQuestionsShort,"<P> Acceding to the throne in 1888 , he dismis...",what happened to germany 's leader after ww1,"[he lost the support of the army , abdicated i...",what happened to [MASK] 's leader after [MASK],"Acceding to the throne in 1888 , he dismissed..."


In [86]:
import joblib
joblib.dump(new_df, "/data/seongilpark/mrqa_df.joblib")

['/data/seongilpark/mrqa_df.joblib']

In [84]:
new_df.to_json("/data/seongilpark/mrqa_masked.json", orient="records", lines=True)

In [85]:
read_data = pd.read_json("/data/seongilpark/mrqa_masked.json", orient="records", lines=True)
read_data

,subset,context,question,answers,masked_q,norm_context
0,SQuAD,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,[Saint Bernadette Soubirous],To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha..."
1,SQuAD,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,[a copper statue of Christ],What is in front of [MASK]?,"Architecturally, the school has a Catholic cha..."
2,SQuAD,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,[the Main Building],[MASK] at [MASK] is beside to which structure?,"Architecturally, the school has a Catholic cha..."
3,SQuAD,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,[a Marian place of prayer and reflection],What is the Grotto at [MASK]?,"Architecturally, the school has a Catholic cha..."
4,SQuAD,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,[a golden statue of the Virgin Mary],What sits on top of [MASK] at [MASK]?,"Architecturally, the school has a Catholic cha..."
...,...,...,...,...,...,...
516678,NaturalQuestionsShort,"<P> In social relationships , an ex ( plural i...",what is the meaning of x girl friend,[a former sexual or romantic partner],what is the meaning of x girl friend,"In social relationships , an ex ( plural is e..."
516679,NaturalQuestionsShort,<P> The film follows two families spending tim...,where does the great outdoors movie take place,[Wisconsin],where does the great outdoors movie take place,The film follows two families spending time o...
516680,NaturalQuestionsShort,"<P> Throughout the novel , Junior shares his d...",where does junior want to go to find hope,[off the rez],where does junior want to go to find hope,"Throughout the novel , Junior shares his drea..."
516681,NaturalQuestionsShort,"<P> Acceding to the throne in 1888 , he dismis...",what happened to germany 's leader after ww1,"[he lost the support of the army , abdicated i...",what happened to [MASK] 's leader after [MASK],"Acceding to the throne in 1888 , he dismissed..."


In [83]:
new_df.("/data/seongilpark/mrqa_norm.tsv", index=False)

AttributeError: 'DataFrame' object has no attribute 'to_tsv'

In [81]:
contexts = list(set(new_df.norm_context.tolist()))
len(contexts)

354604

In [ ]:
Dataset.from_pandas(new_df.drop(columns=["context"]))

In [80]:
new_df[new_df.subset == "NewsQA"].norm_context.tolist()[10]

' One of the Marines shown in a famous World War II photograph raising the U.S. flag on Iwo Jima was posthumously awarded a certificate of U.S. citizenship on Tuesday.The Marine Corps War Memorial in Virginia depicts Strank and five others raising a flag on Iwo Jima.Sgt. Michael Strank, who was born in Czechoslovakia and came to the United States when he was 3, derived U.S. citizenship when his father was naturalized in 1935. However, U.S. Citizenship and Immigration Services recently discovered that Strank never was given citizenship papers.At a ceremony Tuesday at the Marine Corps Memorial -- which depicts the flag-raising -- in Arlington, Virginia, a certificate of citizenship was presented to Strank\'s younger sister, Mary Pero.Strank and five other men became national icons when an Associated Press photographer captured the image of them planting an American flag on top of Mount Suribachi on February 23, 1945.Strank was killed in action on the island on March 1, 1945, less than a 

In [16]:
sub_sub_df = sub_df.drop_duplicates(subset=["masked_q"])
sub_sub_df

,subset,context,question,answers,norm_q,masked_q
0,SQuAD,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,[Saint Bernadette Soubirous],to whom did virgin mary allegedly appear in 18...,To whom did the Virgin Mary allegedly appear i...
1,SQuAD,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,[a copper statue of Christ],what is in front of notre dame main building,What is in front of [MASK]?
2,SQuAD,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,[the Main Building],basilica of sacred heart at notre dame is besi...,[MASK] at [MASK] is beside to which structure?
3,SQuAD,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,[a Marian place of prayer and reflection],what is grotto at notre dame,What is the Grotto at [MASK]?
4,SQuAD,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,[a golden statue of the Virgin Mary],what sits on top of main building at notre dame,What sits on top of [MASK] at [MASK]?
...,...,...,...,...,...,...
584658,NaturalQuestionsShort,"<P> Throughout the novel , Junior shares his d...",where does junior want to go to find hope,[off the rez],where does junior want to go to find hope,where does junior want to go to find hope
584660,NaturalQuestionsShort,"<P> Acceding to the throne in 1888 , he dismis...",what happened to germany 's leader after ww1,"[he lost the support of the army , abdicated i...",what happened to germany s leader after ww1,what happened to [MASK] 's leader after [MASK]
584663,NaturalQuestionsShort,<Table> Michigan State -- Notre Dame football ...,when will notre dame played michigan state again,[2026],when will notre dame played michigan state again,when will [MASK] played [MASK] again
584665,NaturalQuestionsShort,<P> Due to Eric 's departure from the show at ...,who does eric end up with in that 70s show,[Donna],who does eric end up with in that 70s show,who does [MASK] end up with in that [MASK] show


In [23]:
new_df = sub_sub_df.reset_index()

In [24]:
new_df

In [3]:
from nltk.tokenize import sent_tokenize

from src.utils import normalize_answer
import spacy
spacy.prefer_gpu(gpu_id=1)
nlp = spacy.load("en_core_web_trf")

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
sample = train.shuffle().select(range(10000))

In [5]:
batch_size = 500
cnt = 0
spacy_answer_sents = []
for i in tqdm(range(0, len(sample), batch_size)):
    batch = sample[i:i+batch_size]
    docs = nlp.pipe(batch["context"], batch_size=batch_size)
    sents = [[sent.text for sent in doc.sents] for doc in docs]
    answers = batch["answers"]
    for sent, answer in zip(sents, answers):
        sent_idx = -1
        for a in answer:
            if sent_idx != -1:
                break
            for idx, s in enumerate(sent):
                if normalize_answer(a) in normalize_answer(s):
                    sent_idx = idx
                    spacy_answer_sents.append(s)
                    break
        if sent_idx == -1:
            cnt += 1
print(cnt)

 50%|█████     | 5/10 [03:44<03:44, 44.97s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 366.00 MiB. GPU 1 has a total capacty of 23.45 GiB of which 151.69 MiB is free. Process 2202385 has 6.14 GiB memory in use. Including non-PyTorch memory, this process has 17.12 GiB memory in use. Of the allocated memory 13.43 GiB is allocated by PyTorch, and 3.24 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [11]:
sample["question"][:10]

['Jimmy Gilmer & the Fireballs had a hit with "Sugar Shack" & Nancy Sinatra with this sweet place',
 'What club headquarters is located in the city of Lyon, that had a French former footballer who played as a winger or an attacking midfielder pass through the youth training center?',
 'which actor was appealed too?',
 'Little Miss Marker is a 1980 American comedy-drama film written and directed by Walter Bernstein, based on a short story by Damon Runyon, was an American newspaperman, and short story writer, born in which year?',
 'What nationality is racing driver Felipe Massa?',
 'How many times has the singer has been taken to a hospital?',
 'In order for a genocide classification to happen, a major part of a group has to be what?',
 'what was in the letter',
 "(Jimmy of the Clue Crew reads from Temple Church in London.)  In this novel, Langdon's first London stop in his search for the holy grail is here at Temple Church",
 "Corfiotes are natives of this island that isn't far from Al

In [10]:

spacy_answer_sents[:10]

['SUGAR TOWN - Nancy Sinatra View.',
 '[PAR] [TLE] Centre Tola Vologe [SEP] The Centre Tola Vologe is the training center and club headquarters of French football club Olympique Lyonnais.',
 'A group of Nigerian rebels who wrote a letter to U.S. President George W. Bush, stating that they attacked two oil pipelines Monday, have asked for former President Jimmy Carter and actor George Clooney to help solve issues in the oil-rich Niger-delta.',
 '[PAR] [TLE] Damon Runyon [SEP] Alfred Damon Runyon (October 4, 1880 – December 10, 1946) was an American newspaperman and short story writer.',
 '[PAR] Brazilian race car driver Felipe Massa has an estimated net worth of $8 million in 2013.',
 'A long convoy of police and a Los Angeles Fire Department ambulance transported the 26-year-old singer to the hospital after midnight, acting on what the Los Angeles Times reported was a "mental evaluation hold."\n\n\n\nIt was the second hospitalization for Spears this month.',
 'The definition upholds th

In [9]:
nltk_answer_sents[:10]

["Up in your ... i ain't worried 'bout a \nthing cuz i just hit me a lick i gotta fat ... you're just as sweet as can be ... SUGAR \nSHACK - Jimmy Gilmer & The Fireballs ... SUGAR TOWN - Nancy Sinatra View.",
 '[PAR] [TLE] Centre Tola Vologe [SEP] The Centre Tola Vologe is the training center and club headquarters of French football club Olympique Lyonnais.',
 'LAGOS, Nigeria (CNN) -- A group of Nigerian rebels who wrote a letter to U.S. President George W. Bush, stating that they attacked two oil pipelines Monday, have asked for former President Jimmy Carter and actor George Clooney to help solve issues in the oil-rich Niger-delta.',
 '[PAR] [TLE] Damon Runyon [SEP] Alfred Damon Runyon (October 4, 1880 – December 10, 1946) was an American newspaperman and short story writer.',
 '[DOC] [TLE] Felipe Massa Net Worth - TheRicheststumbleupon [PAR] More StatsView More [PAR] About Felipe Massa [PAR] Brazilian race car driver Felipe Massa has an estimated net worth of $8 million in 2013.',
 '

In [8]:
cnt = 0
nltk_answer_sents = []
for row in tqdm(sample):
    answers = row["answers"]
    sents = sent_tokenize(row["context"])
    sent_idx = -1
    for answer in answers:
        if sent_idx != -1:
            break
        for idx, sent in enumerate(sents):
            if normalize_answer(answer.strip().lower()) in normalize_answer(sent.strip().lower()):
                sent_idx = idx
                nltk_answer_sents.append(sent)
                break
    if sent_idx == -1:
        cnt += 1
print(cnt)

100%|██████████| 10000/10000 [00:09<00:00, 1108.70it/s]

42


In [32]:
train = load_dataset("Atipico1/NQ-10k", split="train")
test = load_dataset("Atipico1/NQ", split="test")

In [37]:
dataset = load_dataset("Atipico1/NQ-10k")

Generating test split: 100%|██████████| 3610/3610 [00:00<00:00, 56912.43 examples/s]


In [38]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answers', 'ctxs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['question', 'answers', 'ctxs'],
        num_rows: 3610
    })
})

In [35]:
total = DatasetDict({'train': train, 'test': test})
total.push_to_hub("Atipico1/NQ-10k")

README.md: 100%|██████████| 553/553 [00:00<00:00, 1.90MB/s]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/NQ-10k/commit/f8dc1d1786d9e352eeda939e3a56e142be3457ce', commit_message='Upload dataset', commit_description='', oid='f8dc1d1786d9e352eeda939e3a56e142be3457ce', pr_url=None, pr_revision=None, pr_num=None)

In [113]:
df = pd.read_csv('popQA.tsv', sep='\t')
df.head(1)

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,4222362,George Rankin,occupation,politician,1850297,22,2834605,"[""George James Rankin""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5543720,http://www.wikidata.org/entity/Q82955,George Rankin,Politician,142,25692,What is George Rankin's occupation?,"[""politician"", ""political leader"", ""political ..."


In [114]:
with open("/home/seongilpark/finetune_rag/contriever_results.jsonl", "r") as f:
    results = [json.loads(line) for line in f.readlines()]

In [115]:
pd.DataFrame(results).head(1)

,id,question,answers,ctxs
0,4222362,What is George Rankin's occupation?,"[politician, political leader, political figur...","[{'id': '11701803', 'title': 'George Rankin', ..."


In [116]:
concat = pd.merge(df, pd.DataFrame(results), left_on='id', right_on='id').drop(columns=['question_y', "answers"]).rename(columns={'question_x': 'question', "possible_answers": "answers"})

In [127]:
dataset = Dataset.from_pandas(concat)


In [128]:
dataset = dataset.train_test_split(train_size=10_000, seed=42)

In [132]:
for row in dataset["train"]:
    print(row["ctxs"])
    break

[{'hasanswer': False, 'id': '13995485', 'score': '1.6310263', 'text': 'Escape Artists Escape Artists Productions, LLC, commonly known as Escape Artists and distinct from Escape Artists, Inc. (pod caster), is an independently financed motion picture and television production company with a first look non-exclusive deal at Sony Pictures Entertainment, headed by partners Steve Tisch, Todd Black, and Jason Blumenthal. In 2001, Todd Black and Jason Blumenthal’s Black & Blu merged with the Steve Tisch Company to form Escape Artists. The first produced movie under the Escape Artists banner was "A Knight\'s Tale", starring Heath Ledger in 2001. In the fall of 2005, Escape Artists released "The Weather Man", directed by', 'title': 'Escape Artists'}, {'hasanswer': False, 'id': '18332725', 'score': '1.6303782', 'text': 'Tommy Rettig, and Brian Keith. The announcers were Jack McCoy and Elliott Lewis. Escape (1950 TV series) Escape was a 30-minute live American dramatic anthology television series 

In [129]:
dataset.push_to_hub("Atipico1/popQA")

Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.24s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/popQA/commit/3586e3709ae7947c9629784cfc0705786eec7f0b', commit_message='Upload dataset', commit_description='', oid='3586e3709ae7947c9629784cfc0705786eec7f0b', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
new_dataset["train"]["ctxs"][0]

[{'id': 679735,
  'text': '"Kodoku no Akatsuki" (孤独のあかつき, "The Solitude of Dawn") (also known by its French title "La Solitude de l\'aube") is a song by Japanese musician Ringo Sheena. It was released as one of the A-sides of her 14th single, along with the song "Irohanihoheto", released on May 27, 2013. The release date was the 15th anniversary of the release of Sheena\'s debut single "Kōfukuron" in 1998.',
  'hasanswer': False},
 {'id': 1944639,
  'text': "Doctor Who at the BBC Radiophonic Workshop Volume 1: The Early Years 1963–1969 was the first in a series of compilations of Doctor Who material recorded by the BBC Radiophonic Workshop. Compiled and remastered by Mark Ayres, the album features mostly sound effects and atmospheres from the first six years of the programme. Although some incidental music tracks do appear, most of the album's content is by original Doctor Who sound effects creator Brian Hodgson. The compilation also features three Radiophonic Workshop realisations of 

In [14]:
new_dataset = dataset.train_test_split(test_size=4282, seed=42)

In [16]:
new_dataset.push_to_hub("Atipico1/PopQA")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.65s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/PopQA/commit/08f0f594f1aa7342667bc6ed6c90368d048d3e76', commit_message='Upload dataset', commit_description='', oid='08f0f594f1aa7342667bc6ed6c90368d048d3e76', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
from train import formatting_for_original

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [77]:
akari = load_dataset("akariasai/PopQA")

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [79]:
for row in akari["test"]:
    if not row["possible_answers"]:
        print(row["question"])

In [80]:
akari

DatasetDict({
    test: Dataset({
        features: ['id', 'subj', 'prop', 'obj', 'subj_id', 'prop_id', 'obj_id', 's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title', 'o_wiki_title', 's_pop', 'o_pop', 'question', 'possible_answers'],
        num_rows: 14267
    })
})

In [56]:
import pandas as pd

df_akari = pd.DataFrame(akari["test"])
dataset = load_dataset("Atipico1/PopQA")
df_hf_train = pd.DataFrame(dataset["train"])
df_hf_test = pd.DataFrame(dataset["test"])
new_df_hf_train = pd.merge(df_hf_train, df_akari, on=["id"], how="left")
new_df_hf_train.drop(columns=["question_y"], inplace=True)
new_df_hf_train.rename(columns={"question_x": "question", "possible_answers":"answers"}, inplace=True)
new_df_hf_test = pd.merge(df_hf_test, df_akari, on=["id"], how="left")
new_df_hf_test.drop(columns=["question_y"], inplace=True)
new_df_hf_test.rename(columns={"question_x": "question", "possible_answers":"answers"}, inplace=True)

In [57]:
from datasets import DatasetDict

final = DatasetDict({"train": Dataset.from_pandas(new_df_hf_train), "test": Dataset.from_pandas(new_df_hf_test)})

In [59]:
final.push_to_hub("Atipico1/PopQA")

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.26s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/PopQA/commit/4b229215976ff0e532df636ff94b60119d7cb674', commit_message='Upload dataset', commit_description='', oid='4b229215976ff0e532df636ff94b60119d7cb674', pr_url=None, pr_revision=None, pr_num=None)

In [81]:
final

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'ctxs', 'subj', 'prop', 'obj', 'subj_id', 'prop_id', 'obj_id', 's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title', 'o_wiki_title', 's_pop', 'o_pop', 'answers'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['id', 'question', 'ctxs', 'subj', 'prop', 'obj', 'subj_id', 'prop_id', 'obj_id', 's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title', 'o_wiki_title', 's_pop', 'o_pop', 'answers'],
        num_rows: 4282
    })
})

In [24]:
dataset = load_dataset("Atipico1/popQA")

In [25]:
from ast import literal_eval
dataset["train"] = dataset["train"].map(lambda x: {"answers": literal_eval(x["answers"])})
dataset["test"] = dataset["test"].map(lambda x: {"answers": literal_eval(x["answers"])})

Map:   0%|          | 0/4267 [00:00<?, ? examples/s]

Map: 100%|██████████| 4267/4267 [00:00<00:00, 6164.21 examples/s]


In [31]:
dataset.push_to_hub("Atipico1/popQA")

Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.63s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/popQA/commit/39d1f9de940475fbf588c1d1c49f2fbabb49f477', commit_message='Upload dataset', commit_description='', oid='39d1f9de940475fbf588c1d1c49f2fbabb49f477', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
from src.utils import normalize_question

num_contexts = 5
def formatting_for_original(example):
    output_texts = []
    for i in range(len(example['question'])):
        q = normalize_question(example['question'][i])
        if num_contexts == 0:
            text = f"### Q: {q}\n### A: {example['answers'][i][0]}</s>"
        else:
            ctxs = "\n".join([f"Doc {idx}: {ctx['text']}" for idx, ctx in enumerate(example["ctxs"][i][:num_contexts])])
            text = f"### Background:\n{ctxs}\n\n### Q: {q}\n### A: {example['answers'][i][0]}</s>"
        output_texts.append(text)
    return output_texts
output = formatting_for_original(dataset[:])

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [138]:
out = tokenizer(output, return_length=True)

In [139]:
max(out['length'])

1367

In [36]:
1024+512

1536